In [53]:
import findspark
findspark.init('/home/nowshad/spark-3.3.2-bin-hadoop3/')
from pyspark.sql import SparkSession

In [54]:
spark=SparkSession.builder.appName('lr-example').getOrCreate()

In [55]:
from pyspark.ml.regression import LinearRegression

In [61]:
data=spark.read.csv('/home/nowshad/Downloads/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [62]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [86]:
print(data.head(1)[0])
 

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)


In [73]:
for item in data.head(2)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [76]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [87]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [89]:
assembler=VectorAssembler(inputCols=[ 'Avg Session Length','Time on App','Time on Website','Length of Membership','Yearly Amount Spent'],outputCol='features')

In [90]:
output=assembler.transform(data)

In [91]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [92]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [93]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [94]:
final_data=output.select('features','Yearly Amount Spent')

In [95]:
final_data.show(20)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [122]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [123]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean|  499.4836531365901|
| stddev|  79.04383978608271|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



In [124]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean|  498.9182702099265|
| stddev|  80.20802664712599|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [125]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [126]:
lr_model=lr.fit(train_data)

23/03/19 02:02:29 WARN Instrumentation: [25923cb4] regParam is zero, which might cause numerical instability and overfitting.


In [127]:
lr_model.coefficients

DenseVector([-0.0, -0.0, 0.0, -0.0, 1.0])

In [128]:
lr_model.intercept

2.5096408479138244e-10

In [129]:
test_results=lr_model.evaluate(test_data)

In [130]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-5.34328137291595...|
|1.136868377216160...|
|3.979039320256561...|
|8.697043085703626...|
|1.568878360558301...|
|2.046363078989088...|
|-3.75166564481332...|
|1.364242052659392...|
|-4.14956957683898...|
|-1.70530256582424...|
|7.617018127348274...|
|9.094947017729282...|
|2.728484105318784...|
|1.097077984013594...|
|-4.20641299569979...|
|-2.61479726759716...|
|8.29913915367797E-12|
|9.606537787476555...|
|2.671640686457976...|
|1.705302565824240...|
+--------------------+
only showing top 20 rows



In [131]:
test_results.rootMeanSquaredError

6.51958248748649e-12

In [132]:
test_results.r2

1.0

In [133]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [134]:
unlabeled_data=test_data.select('features')

In [135]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8162006488763...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.2834474760581...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8627411090001...|
+--------------------+
only showing top 20 rows



In [137]:
predictions=lr_model.transform(unlabeled_data)

In [138]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726328|
|[30.5743636841713...|442.06441375806554|
|[30.7377203726281...| 461.7807421962295|
|[30.8162006488763...| 266.0863409484603|
|[31.1239743499119...| 486.9470538397501|
|[31.1280900496166...| 557.2526867470526|
|[31.2834474760581...| 591.7810894256712|
|[31.4252268808548...| 530.7667186547606|
|[31.4459724827577...| 484.8769649351327|
|[31.4474464941278...| 418.6027420952257|
|[31.5741380228732...| 544.4092721605792|
|[31.5761319713222...| 541.2265839893274|
|[31.6610498227460...| 416.3583535798981|
|[31.6739155032749...|475.72506790987023|
|[31.7207699002873...| 538.7749334780272|
|[31.7656188210424...|496.55408163560975|
|[31.8093003166791...| 536.7718993628329|
|[31.8164283341993...|501.12249150364676|
|[31.8279790554652...|440.00274754693885|
|[31.8627411090001...|  556.298141174045|
+--------------------+------------

In [139]:
predictions.head(1)

[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464, 408.6404]), prediction=408.6403510726328)]

In [140]:
test_data.head(1)

[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464, 408.6404]), Yearly Amount Spent=408.6403510726275)]